# Analytic Preparation: 

In [79]:
%%capture
!pip install splunk-sdk
!pip install pandas
!pip install pandasql
import matplotlib.pyplot as plt
import pandas as pd
import pandasql
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 0)

In [80]:
import splunklib.client as client
# Create a Service instance and log in 
service = client.connect(
    host='192.168.1.68', #Input your Splunk's IP here
    port='8089',
    username="admin",
    password="Changeme1!",
    scheme='https')

# Title: Service Creation (Traige)
###  Author: Jonathan Johnson & Jared Atkinson

#### Pulling Data from Detection Notebook:

In [81]:
%store -r Base_Condition

## Context Objective: 
### Registry Key Event &larr; &rarr; Registry Value Set &larr; &rarr; Network Event &larr; &rarr; Process Creation

## Data Collection:

### Registry Set Values

In [82]:
#ImagePath 
from splunklib.searchcommands import *
from tqdm.notebook import tqdm
from time import sleep
import splunklib.results as ImagePath
query = "search index=Windows sourcetype=Sysmon EventCode=13 TargetObject=\"HKLM\\\System\\\CurrentControlSet\\\Services\\\*\\\ImagePath\" earliest=-15m"
query_results = service.jobs.oneshot(query, count=0)
reader = ImagePath.ResultsReader(query_results)

ImagePath = []

for result in reader:
    ImagePath.append(result)
    
df_EID_13_imagepath = pd.DataFrame(ImagePath)    
    
    
if len(df_EID_13_imagepath.index) > 0:
    df2_EID_13_imagepath = df_EID_13_imagepath.Message # Pulling message tag
    df3_EID_13_imagepath = df2_EID_13_imagepath.str.split('\n') #"Splitting data on new-line (\n)"
    df4_EID_13_imagepath = [{a.split(":")[0].strip():":".join(a.split(":")[1:]).strip() for a in b if ":".join(a.split(":")[1:]).strip() != ''} for b in df3_EID_13_imagepath] #Splitting [:] / Removing null values
    Value_Set_ImagePath = pd.DataFrame(df4_EID_13_imagepath) #Convert list to dataframe
if len(df_EID_13_imagepath.index) == 0:
    print ("No Data")

### Pulling Network Events:

In [83]:
#Host Network: 

import splunklib.results as Network
from splunklib.searchcommands import *
from tqdm.notebook import tqdm
from time import sleep

query = "search index=Windows sourcetype=Sysmon EventCode=3 DestinationHostname=*.marvel.local earliest=-15m"
query_results = service.jobs.oneshot(query, count=0)
reader = Network.ResultsReader(query_results)

Network = []

for result in reader:
    Network.append(result)
    
df_EID_3=pd.DataFrame(Network)

if len(df_EID_3.index) > 0:
    df2_EID_3 = df_EID_3.Message # Pulling message tag
    df3_EID_3 = df2_EID_3.str.split('\n') #"Splitting data on new-line (\n)"
    df4_EID_3 = [{a.split(":")[0].strip():":".join(a.split(":")[1:]).strip() for a in b if ":".join(a.split(":")[1:]).strip() != ''} for b in df3_EID_3] #Splitting [:] / Removing null values
    Network_Events = pd.DataFrame(df4_EID_3) #Convert list to dataframe
    Network_Events_update = Network_Events['Image'].str.split('\\').str[-1]
    Network_Events.update(Network_Events_update)
    %store Network_Events
if len(df_EID_3.index) == 0:
    print("No Data")
    %store Network_Events

Stored 'Network_Events' (DataFrame)


### RPC Host Data:

In [84]:
import splunklib.results as RPC
from splunklib.searchcommands import *
from tqdm.notebook import tqdm
from time import sleep

query = "search index=Windows sourcetype=Security EventCode=5156 (Application_Name!=*splunk* AND Application_Name!=System) Direction=Inbound earliest=-15m"
query_results = service.jobs.oneshot(query, count=0)
reader = RPC.ResultsReader(query_results)

RPC = []

for result in reader:
     RPC.append(result) 

df_EID_5156=pd.DataFrame(RPC)

if len(df_EID_5156.index) > 0:
    df2_EID_5156 = df_EID_5156.Message # Pulling message tag
    df3_EID_5156 = df2_EID_5156.str.split('\n') #"Splitting data on new-line (\n)"
    df4_EID_5156 = [{a.split(":")[0].strip():":".join(a.split(":")[1:]).strip() for a in b if ":".join(a.split(":")[1:]).strip() != ''} for b in df3_EID_5156] #Splitting [:] / Removing null values
    RPC_Server = pd.DataFrame(df4_EID_5156) #Convert list to dataframe
    RPC_Server_update = RPC_Server['Application Name'].str.split('\\').str[-1]
    RPC_Server.update(RPC_Server_update)
    %store RPC_Server
if len(df_EID_5156.index) == 0:
    print("No Data")
    %store RPC_Server

Stored 'RPC_Server' (DataFrame)


In [85]:
import splunklib.results as RPC
from splunklib.searchcommands import *
from tqdm.notebook import tqdm
from time import sleep

query = "search index=Windows sourcetype=Security EventCode=5156 (Application_Name!=*splunk*) Direction=Outbound earliest=-15m"
query_results = service.jobs.oneshot(query, count=0)
reader = RPC.ResultsReader(query_results)

RPC = []

for result in reader:
     RPC.append(result)
        
df_EID_5156=pd.DataFrame(RPC)

if len(df_EID_5156.index) > 0:
    df2_EID_5156 = df_EID_5156.Message # Pulling message tag
    df3_EID_5156 = df2_EID_5156.str.split('\n') #"Splitting data on new-line (\n)"
    df4_EID_5156 = [{a.split(":")[0].strip():":".join(a.split(":")[1:]).strip() for a in b if ":".join(a.split(":")[1:]).strip() != ''} for b in df3_EID_5156] #Splitting [:] / Removing null values
    RPC_Client = pd.DataFrame(df4_EID_5156) #Convert list to dataframe
    RPC_Client_update = RPC_Client['Application Name'].str.split('\\').str[-1]
    RPC_Client.update(RPC_Client_update)
    %store RPC_Client
if len(df_EID_5156.index) == 0:
    print("No Data")
    %store RPC_Client

Stored 'RPC_Client' (DataFrame)


### Zeek:

In [86]:
import splunklib.results as Zeek
from splunklib.searchcommands import *
from tqdm.notebook import tqdm
from time import sleep

query = "search index=Zeek source=*dce_rpc.log earliest=-15m"
query_results = service.jobs.oneshot(query, count=0)
reader = Zeek.ResultsReader(query_results)

Zeek = []

for result in reader:
     Zeek.append(result)

Zeek=pd.DataFrame(Zeek)
    
if len(Zeek.index) > 0:
    Zeek=Zeek._raw
    import json
    Zeek = [json.loads(j) for j in Zeek]
    Zeek = pd.DataFrame(Zeek) #Splitting [:] / Removing null values
    %store Zeek
if len(Zeek.index) == 0:
    print("No Data")
    %store Zeek

Stored 'Zeek' (DataFrame)


### Process Creation Events:

In [99]:
import splunklib.results as Process_Creation
from splunklib.searchcommands import *
from tqdm.notebook import tqdm
from time import sleep
#Type: 
query = "search index=Windows sourcetype=Sysmon EventCode=1 earliest=-15m"
query_results = service.jobs.oneshot(query, count=0)
reader = Process_Creation.ResultsReader(query_results)

Process_Creation = []

for result in reader:
    Process_Creation.append(result)

df_EID_1=pd.DataFrame(Process_Creation)

if len(df_EID_1.index) > 0:
    df2_EID_1 = df_EID_1.Message # Pulling message tag
    df3_EID_1 = df2_EID_1.str.split('\n') #"Splitting data on new-line (\n)"
    df4_EID_1 = [{a.split(":")[0].strip():":".join(a.split(":")[1:]).strip() for a in b if ":".join(a.split(":")[1:]).strip() != ''} for b in df3_EID_1] #Splitting [:] / Removing null values
    Process_Creation = pd.DataFrame(df4_EID_1) #Convert list to dataframe
    Process_Creation_Update = Process_Creation['ParentImage'].str.split('\\').str[-1]
    Process_Creation.update(Process_Creation_Update)
    %store Process_Creation
if len(df_EID_1.index) == 0:
    print("No Data")
    %store Process_Creation

Stored 'Process_Creation' (DataFrame)


# JOIN Functions:

### JOINing set values for `ImagePath` with Registry Key Creations

In [88]:
from pandasql import sqldf
Registry_JOIN = pandasql.sqldf(
"""
SELECT 
a.UtcTime,
b.ProcessGuid, 
b.Image,
b.TargetObject,
a.Details
FROM Value_Set_ImagePath a
INNER JOIN Base_Condition b
ON a.UtcTime = b.UtcTime 
"""

).drop_duplicates(subset=['UtcTime']).reset_index().drop("index", axis=1)

In [89]:
%store Registry_JOIN

Stored 'Registry_JOIN' (DataFrame)


In [90]:
Registry_JOIN

,UtcTime,ProcessGuid,Image,TargetObject,Details
0,2020-11-02 18:44:00.629,{63e358a4-2a01-5fa0-0b00-000000002d00},C:\Windows\system32\services.exe,HKLM\System\CurrentControlSet\Services\SO-CON,C:\Windows\System32\notepad.exe


### JOINing Registry Events and Host Network Events (Server): 

In [91]:
from pandasql import sqldf
Registry_Network_JOIN = pandasql.sqldf(
"""
SELECT 
a.UtcTime,
a.ProcessGuid, 
b.Image,
a.TargetObject,
b.User,
b.SourceIp,
b.DestinationIp,
b.SourceHostname,
b.DestinationHostname,
b.SourcePort,
b.DestinationPort, 
a.Details
FROM Registry_JOIN a
INNER JOIN Network_Events b
ON a.ProcessGuid = b.ProcessGuid 
"""

).drop_duplicates(subset=['UtcTime']).reset_index().drop("index", axis=1)

In [92]:
Registry_Network_JOIN_update = Registry_Network_JOIN['Image'].str.split('\\').str[-1]
Registry_Network_JOIN.update(Registry_Network_JOIN_update)
%store Registry_Network_JOIN

Stored 'Registry_Network_JOIN' (DataFrame)


In [93]:
Registry_Network_JOIN

,UtcTime,ProcessGuid,Image,TargetObject,User,SourceIp,DestinationIp,SourceHostname,DestinationHostname,SourcePort,DestinationPort,Details
0,2020-11-02 18:44:00.629,{63e358a4-2a01-5fa0-0b00-000000002d00},services.exe,HKLM\System\CurrentControlSet\Services\SO-CON,NT AUTHORITY\SYSTEM,192.168.146.14,192.168.146.15,ASGARD-WRKSTN,Earth-DC.marvel.local,64505,49680,C:\Windows\System32\notepad.exe


# Isolating Client-Side RPC/Network Data:  
#### This JOIN will seperate the client-side application used to create a service by: 
- Explicitly saying that the application name on the server does not equal the clients application name
- JOINs on `SourcePort`, `DestiatnionPort`, `DestinationIP`, `SourceIP`
- Explicitly saying the network connection is `Outbound`

In [94]:
from pandasql import sqldf
Client_Side_Network_df = pandasql.sqldf(
"""
SELECT 
b.UtcTime,
b.ProcessGuid,
b.Image,
a."Application Name",
b.User,
b.SourceIp,
b.SourceHostname,
b.DestinationHostname,
b.SourcePort,
b.DestinationPort, 
c.endpoint,
c.operation,
b.TargetObject, 
b.Details
FROM RPC_Client  a
JOIN Registry_Network_JOIN b
ON a."Application Name" != b.Image
AND a."Source Port" = b.SourcePort
AND a."Source Address" = b.SourceIp
AND a."Destination Address"=b.DestinationIp
AND a."Destination Port" = b.DestinationPort
AND a.Direction = "Outbound"
JOIN Zeek c
ON c."id.orig_h" = b.SourceIp
AND (c."operation" LIKE "%CreateService%" OR c.operation LIKE "%CreateKey%")
"""

).drop_duplicates().reset_index().drop("index", axis=1)

In [95]:
%store Client_Side_Network_df

Stored 'Client_Side_Network_df' (DataFrame)


### Parent Process

In [100]:
from pandasql import sqldf
Client_Parent_Process_df = pandasql.sqldf(
"""
SELECT 
a.UtcTime,
a.ProcessGuid,
b.ProcessGuid,
b.ParentImage,
a.'Application Name',
a.Image,
a.TargetObject,
a.SourceHostname,
a.DestinationHostname,
a.endpoint,
a.operation,
b.OriginalFilename,
b.Commandline
FROM Client_Side_Network_df a
JOIN Process_Creation b
ON a."Application Name"=b.OriginalFileName

"""

).drop_duplicates().reset_index().drop("index", axis=1)

In [101]:
%store Client_Parent_Process_df

Stored 'Client_Parent_Process_df' (DataFrame)


In [102]:
Client_Parent_Process_df

,UtcTime,ProcessGuid,ProcessGuid,ParentImage,Application Name,Image,TargetObject,SourceHostname,DestinationHostname,endpoint,operation,OriginalFileName,CommandLine
0,2020-11-02 18:44:00.629,{63e358a4-2a01-5fa0-0b00-000000002d00},{FA827E5A-5370-5FA0-9405-000000003000},powershell.exe,sc.exe,services.exe,HKLM\System\CurrentControlSet\Services\SO-CON,ASGARD-WRKSTN,Earth-DC.marvel.local,svcctl,CreateServiceW,sc.exe,"""C:\Windows\system32\sc.exe"" \\192.168.146.15 create SO-CON binpath=C:\Windows\System32\notepad.exe"


# Dataframe Updates:

In [103]:
Parent_Process = []
Command_Line=[]
Iniating_Process = []
Details=[]
Network_Connections=[]
Destination_Host = []
Source_Host = []
Operations = []

In [104]:
for i in Registry_Network_JOIN['Image']:
    Network = i
for j in Value_Set_ImagePath['Details']:
    ImagePath = j 
for l in Client_Side_Network_df['Application Name']:
    Process = l
for l in Client_Parent_Process_df['ParentImage']:
    ParentImage = l
for m in Client_Parent_Process_df['CommandLine']:
    CommandLine = m 
for n in Client_Parent_Process_df['DestinationHostname']:
    DestinationHost = n 
for o in Client_Parent_Process_df['SourceHostname']:
    SourceHost = o 
for p in Client_Parent_Process_df['operation']:
    Operation = p

In [105]:
for a in Base_Condition['UtcTime']:
    Details.append(None)
    Network_Connections.append(None)
    Parent_Process.append(None)
    Command_Line.append(None)
    Iniating_Process.append(None)
    Source_Host.append(None)
    Destination_Host.append(None)
    Operations.append(None)
    for b in Value_Set_ImagePath['UtcTime']:
        if a==b:
            Details[-1] = ImagePath
    for c in Registry_Network_JOIN['UtcTime']:
        if a==c:
            Network_Connections[-1] = Network
    for d in Client_Parent_Process_df['UtcTime']:
        if a==d:
            Parent_Process[-1] = Process
            Command_Line[-1] = CommandLine
            Iniating_Process[-1] = ParentImage
            Destination_Host[-1] = DestinationHost
            Source_Host[-1] = SourceHost
            Operations[-1] = Operation
    
    

In [106]:
Base_Condition['Source Host'] = Source_Host
Base_Condition['Destination Host'] = Destination_Host
Base_Condition['Details'] = Details
Base_Condition['Parent Process'] = Parent_Process
Base_Condition['Iniating Process'] = Iniating_Process
Base_Condition['CommandLine'] = Command_Line
Base_Condition['Network Image'] = Network_Connections
Base_Condition['Operations'] = Operations

In [107]:
Severity=[]
High = 'High'
Medium = 'Medium'
Low = 'Low'

In [108]:
for a, b, c in zip(Base_Condition['EventType'], Base_Condition['Details'], Base_Condition['Network Image']):
    Severity.append(None)
    if a != None and b == None and c == None:
        Severity[-1] = Low
    if a != None and b != None and c == None:
        Severity[-1] = Medium
    if a != None and b != None and c != None:
        Severity[-1] = High

In [109]:
Base_Condition['Severity'] = Severity

In [110]:
%store Base_Condition

Stored 'Base_Condition' (DataFrame)


In [111]:
Base_Condition

,RuleName,EventType,UtcTime,ProcessGuid,ProcessId,Image,TargetObject,Source Host,Destination Host,Details,Parent Process,Iniating Process,CommandLine,Network Image,Operations,Severity
0,-,CreateKey,2020-11-02 18:47:39.477,{FA827E5A-2A19-5FA0-1D00-000000003000},1152,c:\windows\system32\svchost.exe,HKLM\System\CurrentControlSet\Services\NetBT,None,None,None,None,None,None,None,None,Low
1,-,CreateKey,2020-11-02 18:47:15.865,{63e358a4-2a03-5fa0-1a00-000000002d00},1080,C:\Windows\system32\svchost.exe,HKLM\System\CurrentControlSet\Services\NetBT,None,None,None,None,None,None,None,None,Low
2,-,CreateKey,2020-11-02 18:44:00.629,{63e358a4-2a01-5fa0-0b00-000000002d00},552,C:\Windows\system32\services.exe,HKLM\System\CurrentControlSet\Services\SO-CON,ASGARD-WRKSTN,Earth-DC.marvel.local,C:\Windows\System32\notepad.exe,sc.exe,powershell.exe,"""C:\Windows\system32\sc.exe"" \\192.168.146.15 create SO-CON binpath=C:\Windows\System32\notepad.exe",services.exe,CreateServiceW,High
3,-,CreateKey,2020-11-02 18:42:39.441,{FA827E5A-2A19-5FA0-1D00-000000003000},1152,c:\windows\system32\svchost.exe,HKLM\System\CurrentControlSet\Services\NetBT,None,None,None,None,None,None,None,None,Low
4,-,CreateKey,2020-11-02 18:42:15.817,{63e358a4-2a03-5fa0-1a00-000000002d00},1080,C:\Windows\system32\svchost.exe,HKLM\System\CurrentControlSet\Services\NetBT,None,None,None,None,None,None,None,None,Low
5,-,CreateKey,2020-11-02 18:37:39.404,{FA827E5A-2A19-5FA0-1D00-000000003000},1152,c:\windows\system32\svchost.exe,HKLM\System\CurrentControlSet\Services\NetBT,None,None,None,None,None,None,None,None,Low
6,-,CreateKey,2020-11-02 18:37:15.754,{63e358a4-2a03-5fa0-1a00-000000002d00},1080,C:\Windows\system32\svchost.exe,HKLM\System\CurrentControlSet\Services\NetBT,None,None,None,None,None,None,None,None,Low


In [112]:
Final_DF =  Base_Condition

In [113]:
%store Final_DF

Stored 'Final_DF' (DataFrame)


# Alert Email: 

In [32]:
%%capture
import smtplib, ssl
triage = 'http://192.168.1.68:8888/lab/tree/work/Detection%20Notebooks/Service%20Creation%20(T1543.003)/Service_Creation_Triage.ipynb'
investigation = 'http://192.168.1.68:8888/lab/tree/work/Detection%20Notebooks/Service%20Creation%20(T1543.003)/Service_Creation_Investigations.ipynb'
Alert_Name="Service Creation"

for i, (index, row) in enumerate(Base_Condition.iterrows()): 
    time = Base_Condition['UtcTime'][index]
    parent_process=Base_Condition['Parent Process'][index]
    rpc_client=Base_Condition['Iniating Process'][index]
    rpc_server=Base_Condition['Network Image'][index]
    source_host=Base_Condition['Source Host'][index]
    dest_host=Base_Condition['Destination Host'][index]
    targetobject = Base_Condition['TargetObject'][index]
    severity = Base_Condition['Severity'][index]
    commandline=Base_Condition['CommandLine'][index]
    port = 587  # For starttls
    smtp_server = "smtp.gmail.com"
    sender_email = ""
    receiver_email = ""
    password = ""
    message = """Subject: {Alert_Name} ALERT \n
    
    \nSeverity: {severity}
    
    \nA new service was created at time: {time} 
    
    \nService was created on host: {dest_host} 
    
    \nService was created with the registry path: {targetobject}. 
    
    \nThe service request came from host: {source_host}
    
    \nThe process that intialized the service creation was: {rpc_client} 
    
    \nCommandline: {commandline} 
    
    \nParent Process: {parent_process}
    
    \nInvestigation Notebook: {investigation}
    
    \nTriage Notebook: {triage}\n""".format(Alert_Name=Alert_Name, time=time, targetobject=targetobject, triage=triage, investigation=investigation, severity=severity, commandline=commandline, parent_process=parent_process, rpc_client=rpc_client, source_host=source_host,dest_host=dest_host)

    context = ssl.create_default_context()
    try:
        server = smtplib.SMTP(smtp_server,port)
        server.ehlo() # Can be omitted
        server.starttls() # Secure the connection
        server.login(sender_email, password)
        server.sendmail(sender_email, receiver_email, message)
    except Exception as e:
        # Print any error messages to stdout
        print(e)
    finally:
        server.close()

In [33]:
Final_DF.to_pickle('~/work/Final_DF')